You have k lists of sorted integers in non-decreasing order. Find the smallest range that includes at least one number from each of the k lists.

We define the range [a, b] is smaller than range [c, d] if b - a < d - c or a < c if b - a == d - c.

 

Example 1:

Input: nums = [[4,10,15,24,26],[0,9,12,20],[5,18,22,30]]
Output: [20,24]
Explanation: 
list 1: [4, 10, 15, 24,26], 24 is in range [20,24].
list 2: [0, 9, 12, 20], 20 is in range [20,24].
list 3: [5, 18, 22, 30], 22 is in range [20,24].
Example 2:

Input: nums = [[1,2,3],[1,2,3],[1,2,3]]
Output: [1,1]
 

Constraints:

nums.length == k
1 <= k <= 3500
1 <= nums[i].length <= 50
-105 <= nums[i][j] <= 105
nums[i] is sorted in non-decreasing order.

# sliding window:
- expand always, keep track of how many unique syb-array elements are there.
- once ele from all the sub-arrays are there - track and strink to get the min.

In [ ]:
class Solution:
    def smallestRange(self, nums: list[list[int]]) -> list[int]:
        arr = []
        for a in nums:
            arr.extend(a)
        
        arr.sort()
        
        start = 0
        end = 0
        formed_arr = 0
        arr_catch = {}
        ans = []
        r = float('inf')

        for end in range(len(arr)):
            for ind, e in enumerate(nums):
                if arr[end] in e: # NOTE: find which sub-array it belongs too is hard. so put that also in the arr.
                    arr_catch[ind] = arr_catch.get(ind, 0) + 1
            
            while len(arr_catch) == len(nums):
                # We have elements from all the sub-arrys in the window.
                # Track.
                min_range = min(arr[start:end+1])
                max_range = max(arr[start: end+1])
                if max_range - min_range < r:
                    r = max_range - min_range
                    ans = [min_range, max_range]

                # String.
                for ind, e in enumerate(nums):
                    if arr[start] in e:
                        arr_catch[ind] = arr_catch.get(ind, 0) - 1


In [ ]:
from collections import defaultdict

class Solution:
    def smallestRange(self, nums):
        # Step 1: Flatten the lists with (value, list_index)
        arr = []
        for i, lst in enumerate(nums):
            for num in lst:
                arr.append((num, i))
        
        arr.sort()
        
        count = defaultdict(int)
        total_lists = len(nums)
        start = 0
        min_range = float('inf')
        res = [-1, -1]
        unique_lists = 0
        
        for end in range(len(arr)):
            val, idx = arr[end]
            count[idx] += 1
            if count[idx] == 1:  # first number from this list in window
                unique_lists += 1
            
            # Shrink from left if all lists are covered
            while unique_lists == total_lists and start <= end:
                left_val, left_idx = arr[start]

                # Here the end value will be hte max value, satrt value is the min value.
                if val - left_val < min_range:
                    min_range = val - left_val
                    res = [left_val, val]
                
                count[left_idx] -= 1
                if count[left_idx] == 0:
                    unique_lists -= 1
                start += 1
        
        return res
    
# tc:
# arr creation - O(n)
# sorting - O(n log n)
# sliding windoe - O(N)

# sc:
# new arr - O(n)
# count arr - O(#sub-arrys)


In [6]:
Solution().smallestRange(nums = [[4,10,15,24,26],[0,9,12,20],[5,18,22,30]])

[0, 4, 5, 9, 10, 12, 15, 18, 20, 22, 24, 26, 30]


In [ ]:
import heapq

class Solution:
    def smallestRange(self, nums):
        k = len(nums)
        heap = []
        current_max = float('-inf')

        # Step 1: Push the first element of each list
        for i in range(k):
            val = nums[i][0]
            heapq.heappush(heap, (val, i, 0))  # (value, list_index, index_in_list)
            current_max = max(current_max, val)

        min_range = float('inf')
        res = [-1, -1]

        # Step 2: Process heap
        while len(heap) == k:  # All lists must be represented
            current_min, list_idx, ele_idx = heapq.heappop(heap)

            # Update range
            if current_max - current_min < min_range:
                min_range = current_max - current_min
                res = [current_min, current_max]

            # Push next element from the same list
            if ele_idx + 1 < len(nums[list_idx]):
                next_val = nums[list_idx][ele_idx + 1]
                heapq.heappush(heap, (next_val, list_idx, ele_idx + 1))
                current_max = max(current_max, next_val)
            else:
                # One list exhausted, can't find further complete ranges
                break

        return res

# tc - O(n log k) [heap operation]
# sc - O(k)

# min heap 

nums = [
    [4, 10, 15, 24, 26],
    [0, 9, 12, 20],
    [5, 18, 22, 30]
]



| Step | Heap Content (min-heap)    | Current Max | Pop Element | Push Next | New Heap Content           | Current Range | Width | Comment                 |
| ---- | -------------------------- | ----------- | ----------- | --------- | -------------------------- | ------------- | ----- | ----------------------- |
| 1    | (0,1,0),(4,0,0),(5,2,0)    | 5           | -           | -         | same                       | \[0,5]        | 5     | Init heap               |
| 2    | (0,1,0),(4,0,0),(5,2,0)    | 5           | (0,1,0)     | (9,1,1)   | (4,0,0),(5,2,0),(9,1,1)    | \[4,9]        | 5     | pop min, push next      |
| 3    | (4,0,0),(5,2,0),(9,1,1)    | 9           | (4,0,0)     | (10,0,1)  | (5,2,0),(9,1,1),(10,0,1)   | \[5,10]       | 5     | -                       |
| 4    | (5,2,0),(9,1,1),(10,0,1)   | 10          | (5,2,0)     | (18,2,1)  | (9,1,1),(10,0,1),(18,2,1)  | \[9,18]       | 9     | -                       |
| 5    | (9,1,1),(10,0,1),(18,2,1)  | 18          | (9,1,1)     | (12,1,2)  | (10,0,1),(12,1,2),(18,2,1) | \[10,18]      | 8     | -                       |
| 6    | (10,0,1),(12,1,2),(18,2,1) | 18          | (10,0,1)    | (15,0,2)  | (12,1,2),(15,0,2),(18,2,1) | \[12,18]      | 6     | -                       |
| 7    | (12,1,2),(15,0,2),(18,2,1) | 18          | (12,1,2)    | (20,1,3)  | (15,0,2),(18,2,1),(20,1,3) | \[15,20]      | 5     | -                       |
| 8    | (15,0,2),(18,2,1),(20,1,3) | 20          | (15,0,2)    | (24,0,3)  | (18,2,1),(20,1,3),(24,0,3) | \[18,24]      | 6     | -                       |
| 9    | (18,2,1),(20,1,3),(24,0,3) | 24          | (18,2,1)    | (22,2,2)  | (20,1,3),(22,2,2),(24,0,3) | \[20,24]      | 4     | **Best range found**    |
| 10   | (20,1,3),(22,2,2),(24,0,3) | 24          | (20,1,3)    | -         | -                          | -             | -     | list 1 exhausted → stop |
